In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator, load_disambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['UFO']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [45]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms(weight_decay_param=0.00001)
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > -1]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [46]:
list(zip(longforms, counts))

[('unusual floral organs', 14),
 ('uncleaved prefusion optimized', 2),
 ('ufo', 1),
 ('used frying oil', 1),
 ('unsual floral organs', 1)]

In [6]:
try:
    disamb = load_disambiguator(shortforms[0])
    for shortform, gm in disamb.grounding_dict.items():
        for longform, grounding in gm.items():
            grounding_map[longform] = grounding
    for grounding, name in disamb.names.items():
        names[grounding] = name
    pos_labels = disamb.pos_labels
except Exception:
    pass

In [7]:
names

{'HGNC:905': 'AXL'}

In [48]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8891)

In [49]:
result = [grounding_map, names, pos_labels]

In [50]:
result

[{'ufo': 'ungrounded',
  'uncleaved prefusion optimized': 'ungrounded',
  'unsual floral organs': 'ungrounded',
  'unusual floral organs': 'ungrounded',
  'used frying oil': 'ungrounded'},
 {},
 []]

In [123]:
grounding_map, names, pos_labels = [{'ufo': 'HGNC:905',
  'uncleaved prefusion optimized': 'ungrounded',
  'unsual floral organs': 'ungrounded',
  'unusual floral organs': 'ungrounded',
  'used frying oil': 'ungrounded'},
 {},
 ['HGNC:905']]

In [124]:
excluded_longforms = []

In [125]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [126]:
additional_entities = {'HGNC:905': ['AXL', ['UFO']]}

In [127]:
unambiguous_agent_texts = {}

In [128]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [129]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [130]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [131]:
intersection1

[('HGNC:905', 'HGNC:905', 819)]

In [132]:
intersection2

[('ungrounded', 'HGNC:905', 0), ('HGNC:905', 'HGNC:905', 0)]

In [133]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts[1]:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    _, contains = additional_entities[entity]
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=contains)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])

In [134]:
names.update({key: value[0] for key, value in additional_entities.items()})
names.update({key: value[0] for key, value in unambiguous_agent_texts.items()})
pos_labels = list(set(pos_labels) | additional_entities.keys() |
                  unambiguous_agent_texts.keys())

In [135]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-11-10 05:33:15] /adeft/PP/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-11-10 05:33:15] /adeft/PP/adeft/adeft/modeling/classify.py - Best f1 score of 0.9866666666666667 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [136]:
classifier.stats

{'label_distribution': {'ungrounded': 17, 'HGNC:905': 31},
 'f1': {'mean': 0.986667, 'std': 0.026667},
 'precision': {'mean': 0.975, 'std': 0.05},
 'recall': {'mean': 1.0, 'std': 0.0},
 'HGNC:905': {'f1': {'mean': 0.986667, 'std': 0.026667},
  'pr': {'mean': 1.0, 'std': 0.0},
  'rc': {'mean': 0.975, 'std': 0.05}},
 'ungrounded': {'f1': {'mean': 0.96, 'std': 0.08},
  'pr': {'mean': 0.933333, 'std': 0.133333},
  'rc': {'mean': 1.0, 'std': 0.0}}}

In [137]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [138]:
disamb.dump(model_name, results_path)

In [139]:
print(disamb.info())

Disambiguation model for UFO

Produces the disambiguations:
	AXL*	HGNC:905

Class level metrics:
--------------------
Grounding	Count	F1     
AXL*	31	0.98667
Ungrounded	17	   0.96

Weighted Metrics:
-----------------
	F1 score:	0.98667
	Precision:	0.975
	Recall:		1.0

* Positive labels
See Docstring for explanation



In [140]:
model_to_s3(disamb)

In [141]:
preds = [disamb.disambiguate(text) for text in all_texts.values()]

In [142]:
texts = [text for pred, text in zip(preds, all_texts.values()) if pred[0] == 'HGNC:905']

In [144]:
texts[2]

'In the demyelinated and remyelinated corpus callosum, we detected 1239 differentially expressed genes; 91 orthologues were also differentially expressed in MS lesions. Pathway analysis of these orthologues suggested that the TYROBP (DAP12)-TREM2 pathway, TNF-receptor 1, CYBA and the proteasome subunit PSMB9 were related to de- and remyelination. We designed 129 peptides representing 51 orthologous proteins, measured them by PRM in 97 individual CSF, and compared their levels between relapsing (n = 40) and progressive MS (n = 57). Four proteins were differentially regulated among relapsing and progressive MS: tyrosine protein kinase receptor UFO (UFO), TIMP-1, apolipoprotein C-II (APOC2), and beta-2-microglobulin (B2M). The orthologous genes/proteins in the mouse brain peaked during acute remyelination. UFO, TIMP-1 and B2M levels correlated inversely with inflammation in the CSF (IL-6, MCP-1/CCL2, TARC/CCL17). APOC2 showed positive correlation with IL-2, IL-16 and eotaxin-3/CCL26.\nPat